#### Building a Simple LLM application with LCEL

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [4]:
from langchain_groq import ChatGroq

model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002D398FC66B0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002D398FC56F0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [7]:
result

AIMessage(content='Here are a few ways to translate "Hello, how are you?" to French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal, to one person)\n* **Salut, comment allez-vous ?** (Informal, to multiple people)\n\n\nLet me know if you\'d like to know the appropriate level of formality for a specific situation!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 21, 'total_tokens': 112, 'completion_time': 0.165454545, 'prompt_time': 0.000483358, 'queue_time': 0.04691414, 'total_time': 0.165937903}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fc750b48-66a4-4181-ac81-55f851b871ad-0', usage_metadata={'input_tokens': 21, 'output_tokens': 91, 'total_tokens': 112})

In [8]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Here are a few ways to translate "Hello, how are you?" to French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal, to one person)\n* **Salut, comment allez-vous ?** (Informal, to multiple people)\n\n\nLet me know if you\'d like to know the appropriate level of formality for a specific situation!\n'

In [9]:
### Chaining the components using LCEL
chain=model|parser

chain.invoke(messages)

'Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal)\n* **Coucou, ça va ?** (Very informal)\n\n\nThe best option depends on the level of formality you want to convey. \n'

In [10]:
### Prompt templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",generic_template),
        ("user","{text}")
    ]
)

In [11]:
result=prompt.invoke({"language":"French", "text":"Hello"})

In [13]:
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [14]:
chain=prompt|model|parser
chain.invoke({"language":"French", "text":"Hello"})

'Hello in French is **Bonjour**. \n'